# 2.1.2 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

### Schema Nutzungsschablone

In [2]:
# Zeichenerklärung – Schema Nutzungsschablone WITHOUT BOUNDING BOX
# SKIP: ze_raw_path
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE.png"
ze_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"

instruction = 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'

async def run():
    return await ava.extractTextFromImages(instruction, [ze_path, ze_crop_path])
    
results = await runner.async_consistency_check(run)

['1. Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter "Art der Nutzung".

2. Die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone" lautet:
   - 1.5 (§ 9 (1) 1 BauGB)

3. ASCII-Tabellenform:

```
+-------------------+-----------------------------+
| Nutzungsschablone | 1.5 (§ 9 (1) 1 BauGB)       |
+-------------------+-----------------------------+
```', '1. Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter "Planungsrechtliche Festsetzungen".

2. Die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone" lautet:
   - Nutzungsschablone (§ 9 (1) 1 BauGB)

3. Die Tabellenerklärung in ASCII-Tabellenform:

```
+-------------------+-----------------------------+
| Nutzungsschablone | (§ 9 (1) 1 BauGB)            |
+-------------------+-----------------------------+
```']
#############################################
['1. Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter "Art der Nutzung".

2. Die 

In [4]:
# Zeichenerklärung – Schema Nutzungsschablone WITH BOUNDING BOX
ze_bb_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-bb.png"
ze_crop_bb_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-bb-crop.png"

instruction = 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'

async def run():
    return await ava.extractTextFromImages(instruction, [ze_bb_path, ze_crop_bb_path])
    
results = await runner.async_consistency_check(run)

['Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+----------------+----------------+
| Art der Nutzung| Gebäudeklasse  |
+----------------+----------------+
| Grundflächenzahl| Geschossflächen-|
| (GRZ)          | zahl (GFZ)     |
+----------------+----------------+
```', 'Hier ist die rot umrandete Tabellenerklärung zum Eintrag "Nutzungsschablone" in ASCII-Tabellenform:

```
+-------------------+-------------------+
| Art der Nutzung   | Gebäudehöhe       |
+-------------------+-------------------+
| Grundflächenzahl  | Geschossflächen-  |
|                   | zahl              |
+-------------------+-------------------+
| Bauweise          |                   |
+-------------------+-------------------+
```']
#############################################
['Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+----------------+----------------+
| Art der Nutzung| Gebäudeklasse  |


In [6]:
# Zeichenerklärung – Schema Nutzungsschablone CROPPED
ze_nz_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-nz.png"

instruction = 'Gebe das Schema der Tabellenerklärung (Nutzungsschablone) strukturiert in ASCII-Tabellenform aus.'

async def run():
    return await ava.extractTextFromImages(instruction, [ze_nz_path])
    
results = await runner.async_consistency_check(run)

# Store Schema
msg211_f11_nz = results[0][0]
%store msg211_f11_nz 

['Hier ist das Schema der Tabellenerklärung in ASCII-Tabellenform:

```
+-------------------+-------------------+
| Art der Nutzung   | Gebäudehöhe       |
+-------------------+-------------------+
| Grundflächenzahl  | Geschossflächen-  |
|                   | zahl              |
+-------------------+-------------------+
| Bauweise          | -                 |
+-------------------+-------------------+
```']
#############################################
['Hier ist das Schema der Tabellenerklärung (Nutzungsschablone) in ASCII-Tabellenform:

```
+------------------+------------------+
| Art der Nutzung  | Gebäudehöhe      |
+------------------+------------------+
| Grundflächenzahl | Geschossflächen- |
|                  | zahl             |
+------------------+------------------+
| Bauweise         | -                |
+------------------+------------------+
```']
#############################################
['Hier ist das Schema der Tabellenerklärung in ASCII-Tabellenform:

```
+---

### Zeichenerklärung der Nutzungsschablone

In [5]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien – NO CONTEXT
images = [ze_path, ze_crop_path]

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

messages = await ava.extractTextFromImages(instruction, images)
pprint(messages)

Hier sind die extrahierten Informationen aus dem Bebauungsplan zu den angeforderten Themen:

### Art der baulichen Nutzung
- **GE**: Gewerbegebiet (§ 8 BauNVO)
- **GEe**: Gewerbegebiet eingeschränkt (§ 8 BauNVO)
- **Grünfläche**: 
  - Öffentliche Grünflächen (§ 9 (1) 15 BauGB)
  - Grünflächen mit Bindungen für die Erhaltung von Bäumen, Sträuchern und sonstigen Bepflanzungen (§ 9 (1) 25c BauGB)
  - Grünflächen mit Bindungen für Maßnahmen zum Schutz, zur Pflege und zur Entwicklung von Natur und Landschaft (§ 9 (1) 20 BauGB)
  - Grünflächen mit Bindungen zur Anpflanzung von Bäumen, Sträuchern und sonstigen Bepflanzungen (§ 9 (1) 25a BauGB)
  - Grünflächen mit Bindungen zur Anpflanzung von Bäumen, Sträuchern und sonstigen Bepflanzungen (§ 9 (1) 25b BauGB)
- **LR1**: Leitungsrecht zu Gunsten der EVS / Strom
- **LR2**: Leitungsrecht zu Gunsten der EVS / Siel / Strom und Wasser

### Maß der baulichen Nutzung
- **GRZ**: Grundflächenzahl (§ 16 BauNVO)
  - 0,6
  - 1,2
- **GFZ**: Geschossflächenz

In [7]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien – WITH CONTEXT
# Kontext: Schema, BauNVO
%store -r msgbaunvo
images = [ze_path, ze_crop_path]
schemas = parser.text2prompts([msg211_f11_zebb_nz, msg211_f11_zecropbb_nz])
baunvo = parser.text2prompts([msgbaunvo])
contexts = list(map(lambda schema: [*baunvo,schema], schemas))

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

message = await ava.extractTextFromImagesWithContexts(instruction, images, contexts)
pprint(message)

### Art der baulichen Nutzung

- **GE**: Gewerbegebiet (§ 8 BauNVO)
- **GEe**: Gewerbegebiet eingeschränkt (§ 8 BauNVO)
- **Nutzungsschablone**: § 9 (1) und (2) BauGB

### Maß der baulichen Nutzung

- **max. zulässige Gebäudehöhe**: 12,00 m über Erdgeschossfußbodenhöhe
- **Grundflächenzahl (GRZ)**: 0,6 (§ 19 BauNVO)
- **Geschossflächenzahl (GFZ)**: 1,2 (§ 20 BauNVO)

### Bauweise, überbaubare Grundstücksfläche

- **Bauweise**: Offene Bauweise (§ 22 BauNVO)
- **Baugrenze**: § 23 (3) BauNVO
- **Überbaubare Grundstücksfläche**: § 23 BauNVO

### Weitere Festsetzungen

- **Umgrenzung von Flächen, die von der Bebauung freizuhalten sind**: § 9 (1) 10 BauGB
- **Versorgungsflächen**: § 9 (1) 12 BauGB
- **Rad-/Gehweg, landwirtschaftlicher Verkehr**: § 9 (1) 9 BauGB
- **Bereich ohne Ein- und Ausfahrt**: § 9 (1) 11 BauGB
- **Untersuchung**: § 9 (1) 15 BauGB
- **Öffentliche Grünflächen**: § 9 (1) 15 BauGB
- **Umgrenzung von Flächen für Maßnahmen zum Schutz, zur Pflege und zur Entwicklung von Natur 